<a href="https://colab.research.google.com/github/Md-Jahid-Hasan/Data-Science-AI-ML/blob/master/TorchDatasetCollector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io

In [ ]:
class CollectData(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.data = self.get_data()
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.directory, self.data.iloc[index, 0])
        image = io.imread(img_path)
        label = torch.tensor(int(self.data.iloc[index, 1]))
        
        return (image, label)
        
        
    
    def get_data(self):
        create_datset = True
        labels = []
        path = []
        
        for id, folder_name in tqdm(enumerate(os.scandir(self.directory))):
            print(id)
            if folder_name.name == "dataset.csv":
                create_datset = False

            if folder_name.is_dir():
                for file in os.listdir(folder_name):
                    path.append(f"{folder_name.name}\{file}")
                    labels.append(id)


        if create_datset:
            data = {"image":path, "label":labels}
            df = pd.DataFrame(data)
            df.to_csv(f"{self.directory}\dataset.csv", header=False, index=False)
            
        return pd.read_csv(f"{self.directory}\dataset.csv")
        
        
        
    

In [ ]:
class PrepareTorchData():
    def __init__(self, directory):
        self.directory = directory
        self.dataset = CollectData(self.directory)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        return self.dataset[index]
    
    def save_torch(self):
        train_set, test_set = torch.utils.data.random_split(dataset, [20001, 5000])
        train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
        test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=True)
        torch.save(train_loader, os.path.join(self.directory,"train_loader.pt"))
        torch.save(test_loader, os.path.join(self.directory,"test_loader.pt"))
        print(f"Train and Test Loader save in {self.directory} this directory")